In [ ]:
import skiba.data_process as sdp
import ipywidgets as widgets
import geopandas as gpd
import geemap as gm
import requests
import json
import os

In [ ]:
m = sdp.data_process
dropdown_widget = m.create_dropdown()
dropdown_widget

In [ ]:
import ee

ee.Initialize(project="ee-forestplotvariables")
ee.Authenticate()

In [ ]:
m1 = sdp.data_process
start = m1.add_date_picker()
start

In [ ]:
end = m1.add_date_picker()
end

In [ ]:
points = "../data/selected_coordinates.csv"
geedata = dropdown_widget.value
end_date = end.value
start_date = start.value

In [ ]:
final = sdp.data_process
returned_dataset = final.get_coordinate_data(
    data=points, geedata=geedata, start_date=start_date, end_date=end_date
)

# Extra testing

In [ ]:
url = (
    "https://github.com/opengeos/geospatial-data-catalogs/blob/master/gee_catalog.json"
)

In [ ]:
# def fetch_geojson(url):
#     try:
#         response = requests.get(url)
#         response.raise_for_status()  # Raises an exception for HTTP errors
#         geojson_data = response.json()  # Parse the JSON response
#         return geojson_data
#     except requests.exceptions.HTTPError as http_err:
#         print(f"HTTP error occurred: {http_err}")
#     except requests.exceptions.ConnectionError as conn_err:
#         print(f"Error connecting to the server: {conn_err}")
#     except Exception as err:
#         print(f"An error occurred: {err}")
#     return None

# geojson_data = fetch_geojson(url)

In [ ]:
data = "../data/gee_catalog.json"
with open(data, "r") as file:
    data = json.load(file)

data_dict = {item["title"]: item["id"] for item in data if "title" in item}

# Step 4: Create the dropdown
dropdown = widgets.Dropdown(
    options=data_dict,  # keys shown, values returned
    description="Dataset:",
    disabled=False,
)

display(dropdown)

In [ ]:
dropdown.value

In [ ]:
gee = sdp.data_process
geeimage = gee.load_gee_as_image(dropdown_widget.value, start_date, end_date)

In [ ]:
print(geeimage)

In [ ]:
import pandas as pd
import geopandas as gpd

if isinstance(points, str):
    coordinates = pd.read_csv(points)
    gdf = gpd.GeoDataFrame(
        coordinates,
        geometry=gpd.points_from_xy(coordinates.LON, coordinates.LAT),
        crs="EPSG:4326",  # Directly set CRS during creation
    )
else:
    gdf = data.to_crs(epsg=4326)  # Ensure WGS84

geojson = gdf.__geo_interface__
fc = gm.geojson_to_ee(geojson)

In [ ]:
sampled_data = geeimage.sampleRegions(
    collection=fc,  # Adjust scale based on dataset resolution
    geometries=True,  # Include geometry in the output
)

In [ ]:
dropdown_widget.value

In [ ]:
name = dropdown_widget.value
file_name = name.replace("/", "_")

out_dir = os.path.join(os.path.expanduser("~"), "Downloads")
output_file = f"{file_name}.csv"
out_path = os.path.join(out_dir, output_file)

sampled_data_2 = gm.extract_values_to_points(fc, geeimage, out_path)

In [ ]:
final = sdp.data_process

returned_dataset = final.get_coordinate_data(
    data=points, geedata=geedata, start_date=start_date, end_date=end_date
)
output_file = f"{geedata}.csv"
out_dir = os.path.join(os.path.expanduser("~"), "Downloads")

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

out_path = os.path.join(out_dir, output_file)

# gm.ee_to_csv(returned_dataset, filename=output_file, out_dir=out_dir)
# print(f"Data saved to {out_path}")
# make output that says success/failed and where file is saved to